In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

***
# Ayudantía 07
## ACUS099- Procesamiento digital de señales
### 7 de Mayo
#####  Spectral Density
Profesor: Dr. Victor Poblete R.

Ayudante: Diego Espejo A.
***

### Cortando en Frames: Ventaneo


La transformada discreta de Fourier en tiempo corto (STDFT), se realiza por trozos del audio a analizar, estos trozos normalmentes son traslapados unos con otros para reducir irregularidades en la frontera.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import matplotlib
import librosa as lib
import numpy as np
import pandas as pd
from scipy import signal, fftpack
%matplotlib notebook

#### Ejemplo Basico

Tenemos un vector de 1 a 10 el cual deseamos realizarle una STDFT, para eso le realizamos un ventaneo con un traslape al 50% lo cual se traduce en lo siguiente

In [ ]:
A = np.arange(1, 11)
A

![texto_alternativo_imagen](Whiteboard.png)

#### Ejemplo con una señal real

##### Ingreso de señal

In [ ]:
sig, fs = lib.load('MON_20200109_070000.wav',
                   sr=44100, offset=140, duration=15)

##### Fijar valores para ventaneo

In [ ]:
N = 2048
tras_percent = 0.5
n_tras = int(0.5*N)
n_ventanas = int(np.around(len(sig)/(N-n_tras), 0))

##### Zero padding

In [ ]:
zero_pad = n_tras*(n_ventanas+1)
sig_pad = np.zeros(zero_pad)
for i in range(len(sig)):
    sig_pad[i] = sig[i]

##### Ventaneo

In [ ]:
m = np.zeros((N, n_ventanas))
for i in range(n_ventanas):
    m[:, i] = sig_pad[i*n_tras:N+n_tras*i]

#####  Hamming

In [ ]:
hamm = signal.hamming(N)  # hamming o hanning o kaiser
for j in range(n_ventanas):
    m[:, j] = m[:, j]*hamm

#####  FFT

In [ ]:
def mag_cuad(x):
    A = fftpack.fft(m[:, x], N)
    freq = np.linspace(0, fs/2, len(A)//2)
    response = 1.0/N * np.abs(A[:N//2])**2
    db_response = 10*np.log10(response)
    return freq, db_response


def response_peaks(freq, db_response):
    peaks, _ = signal.find_peaks(db_response)
    freq_peaks = np.zeros(len(peaks))
    db_peaks = np.zeros(len(peaks))
    for i in range(len(peaks)):
        v = peaks[i]
        freq_peaks[i] = freq[v]
        db_peaks[i] = db_response[v]
    return freq_peaks, db_peaks

In [ ]:
def plot_sd(x):

    freq, db_response = mag_cuad(x)
    freq_peaks, db_peaks = response_peaks(freq, db_response)
    fig, ax = plt.subplots(figsize=(9, 4), tight_layout=True)
    ax.plot(freq, db_response)
    ax.plot(freq_peaks, db_peaks, 'x')
    ax.set_title('Frame $'+str(x+1)+'$')
    ax.set_ylabel('Magnitud Cuadratica (dB)')
    ax.set_xlabel('Frecuencia (Hz)')
    ax.set_xlim(20, 20000)
    ax.set_xscale('symlog')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    ax.set_xticks([31, 63, 125, 250, 500, 1000, 2000, 4000, 8000, 16000])
    ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    ax.grid('on')
    fig.show()

In [ ]:
interact(plot_sd, x=widgets.IntSlider(
    min=0, max=n_ventanas-1, step=1, value=100))